In [4]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("하이닉스 임시.csv", thousands=',', encoding = 'cp949')

df['next_day_return'] = df['종가_등락률'].shift(-1)
df['target'] = df['next_day_return'].apply(lambda x: 3 if x > 2 else (2 if x > -2 else 1))
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

df1= df.iloc[1:300, :]
X = df1.drop(['날짜','next_day_return','target'], axis=1)
y = df1['target']

In [5]:
df1

,날짜,시가,고가,저가,종가,거래량,MACD,MACD_Signal,RSI,RSI_Signal,...,[일]은행_수량,[일]투신_수량,[일]보험_수량,[일]금융투자_수량,[일]기관_수량,[일]외국인_수량,[일]개인_수량,[일]프로그램순매수(종목),next_day_return,target
1,2023-03-22,85200,87300,83800,86900,2667268,-1619.800495,-1584.762485,47.560976,36.279787,...,-3296,97497,7634,74400,345975,389364,-727842,345117,1.841197,2
2,2023-03-23,85400,88500,85100,88500,3091673,-1182.891704,-1504.388328,52.362205,39.496271,...,-1459,160899,49444,266626,798609,-204360,-592656,-452629,-1.355932,2
3,2023-03-24,89100,90700,86700,87300,4670908,-922.830495,-1388.076762,45.041322,40.605281,...,4818,38074,46836,79821,315758,-650758,315529,259766,-2.061856,1
4,2023-03-27,87500,87800,84700,85500,3211190,-852.151993,-1280.891808,43.253968,41.135019,...,4516,-28201,19023,191967,-31812,-372493,426142,173628,3.391813,3
5,2023-03-28,85300,88700,85200,88400,3180431,-555.726933,-1135.858833,53.076923,43.523399,...,-983,64015,587,217480,642458,-12785,-636771,-292297,-1.696833,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2024-03-13,168600,169700,162000,163600,5072935,6429.682304,6338.987753,62.586207,66.293971,...,925,-730507,-17995,-12383,-973009,-225674,1173688,563429,-1.039120,2
241,2024-03-14,163600,164200,161100,161900,3655060,5900.832235,6251.356649,55.172414,64.069660,...,-1483,-180322,-31041,188945,-134265,-418025,557001,-67782,-0.432366,2
242,2024-03-15,159600,162900,158600,161200,4099531,5363.404606,6073.766241,49.791667,61.214061,...,4198,-28012,-8568,14314,-48164,-113095,170734,-351249,1.923077,2
243,2024-03-18,161200,164500,161000,164300,2696598,5128.515101,5884.716013,52.465483,59.464345,...,-799,87517,2948,471206,679279,-575769,-120655,-88600,-2.495435,1


In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("하이닉스 임시.csv", thousands=',', encoding = 'cp949')

df['next_day_return'] = df['종가_등락률'].shift(-1)
df['target'] = df['next_day_return'].apply(lambda x: 3 if x > 2 else (2 if x > -2 else 1))
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

df1= df.iloc[1:300, :]
X = df1.drop(['날짜','next_day_return','target'], axis=1)
y = df1['target']
# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

#Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# Combining all the data into a single dataframe without merging by feature
final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
                      features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
X_top8 = X[top_8_features]
y = df1['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.3, random_state=42)

# 모델 초기화
models = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
}

# 점수를 저장할 딕셔너리
scores = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': []
}

# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores['Model'].append(model_name)
    scores['Accuracy'].append(accuracy_score(y_test, y_pred))
    scores['Precision'].append(precision_score(y_test, y_pred, average='macro'))
    scores['Recall'].append(recall_score(y_test, y_pred, average='macro'))
    scores['F1_Score'].append(f1_score(y_test, y_pred, average='macro'))

# 점수 딕셔너리를 데이터프레임으로 변환
scores_df = pd.DataFrame(scores)
scores_df

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010817327334812887, tolerance: 0.005499487179487179
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05389051264209854, tolerance: 0.005499487179487179
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05388455747121412, tolerance: 0.005499487179487179
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn

,Model,Accuracy,Precision,Recall,F1_Score
0,SVC,0.770270,0.256757,0.333333,0.290076
1,LogisticRegression,0.756757,0.369327,0.349415,0.330903
2,DecisionTreeClassifier,0.648649,0.349650,0.346491,0.347857
3,RandomForestClassifier,0.702703,0.309287,0.326023,0.311231


In [7]:
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("하이닉스 임시.csv", thousands=',', encoding = 'cp949')

df['next_day_return'] = df['종가_등락률'].shift(-1)
df['target'] = df['next_day_return'].apply(lambda x: 3 if x > 2 else (2 if x > -2 else 1))
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

df1= df.iloc[1:300, :]
X = df1.drop(['날짜','next_day_return','target'], axis=1)
y = df1['target']
# Calculating correlations for Forward Selection
correlations = X.corrwith(y).abs().sort_values(ascending=False).reset_index()
correlations.columns = ['Feature', 'Correlation']

#Performing T-tests
t_tests = {column: ttest_ind(X[column][y == 1], X[column][y == -1], nan_policy='omit') for column in X.columns}
t_tests_sorted = sorted(t_tests.items(), key=lambda x: x[1].pvalue)
t_tests_df = pd.DataFrame(t_tests_sorted, columns=['Feature', 'T-test'])
t_tests_df['T-test'] = t_tests_df['T-test'].apply(lambda x: x.pvalue)  # Only keep p-value for simplicity

# Lasso and Ridge regression
lasso = LassoCV().fit(X, y)
ridge = RidgeCV().fit(X, y)

lasso_importance = np.abs(lasso.coef_)
ridge_importance = np.abs(ridge.coef_)

features_lasso_sorted = sorted(zip(X.columns, lasso_importance), key=lambda x: x[1], reverse=True)
features_ridge_sorted = sorted(zip(X.columns, ridge_importance), key=lambda x: x[1], reverse=True)

features_lasso_df = pd.DataFrame(features_lasso_sorted, columns=['Feature', 'Lasso Importance'])
features_ridge_df = pd.DataFrame(features_ridge_sorted, columns=['Feature', 'Ridge Importance'])

# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# Combining all the data into a single dataframe without merging by feature
final_df = pd.concat([correlations, t_tests_df.drop('Feature', axis=1), features_lasso_df.drop('Feature', axis=1), 
                      features_ridge_df.drop('Feature', axis=1), features_rf_df.drop('Feature', axis=1)], axis=1)

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_8_features = correlations.sort_values(by='Correlation', ascending=False).head(8)['Feature']

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)
X_top8 = X[['[일]프로그램순매수(종목)']]
y = df1['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.3, random_state=42)

# 모델 초기화
models = {
    'SVC': SVC(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
}

# 점수를 저장할 딕셔너리
scores = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1_Score': []
}

# 각 모델에 대해 학습 및 평가
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    scores['Model'].append(model_name)
    scores['Accuracy'].append(accuracy_score(y_test, y_pred))
    scores['Precision'].append(precision_score(y_test, y_pred, average='macro'))
    scores['Recall'].append(recall_score(y_test, y_pred, average='macro'))
    scores['F1_Score'].append(f1_score(y_test, y_pred, average='macro'))

# 점수 딕셔너리를 데이터프레임으로 변환
scores_df = pd.DataFrame(scores)
scores_df

c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010817327334812887, tolerance: 0.005499487179487179
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05389051264209854, tolerance: 0.005499487179487179
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05388455747121412, tolerance: 0.005499487179487179
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn

,Model,Accuracy,Precision,Recall,F1_Score
0,SVC,0.770270,0.256757,0.333333,0.290076
1,LogisticRegression,0.770270,0.256757,0.333333,0.290076
2,DecisionTreeClassifier,0.662162,0.358333,0.369298,0.356125
3,RandomForestClassifier,0.662162,0.358333,0.369298,0.356125
